In [1]:
from tensorflow.keras.datasets import mnist 
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential , Model 
from tensorflow.keras.optimizers import Adam 
import matplotlib.pyplot as plt 
import numpy as np 

In [2]:
#Define the image input Dimensions 
img_row = 28 
img_cols = 28
channels = 1 #Channel=3 for RGB images 
img_shape = (img_row,img_cols,channels)

In [11]:
def build_generator():
    
    noise_shape=(100,) # 1D array of size 100 (laten vector/ noise)
    
    #Here we are only using Dense layers. But network can be complicated based
    #on the application. For example, you can use VGG for super res. GAN.  
    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    
    model.add(Dense(np.prod(img_shape), activation = 'tanh'))
    model.add(Reshape(img_shape))
    
    model.summary()
    
    noise = Input(shape=noise_shape)
    Fake_img = model(noise)
    
    
    return Model(noise,Fake_img)

In [12]:
def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img) ##Real images

    return Model(img, validity)

In [13]:
def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [24]:
def train(epochs, batch_size=128, save_interval=50):

    (X_train, _), (_, _) = mnist.load_data()

    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    
    X_train = np.expand_dims(X_train, axis=3) 

    half_batch = int(batch_size / 2)
    
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        imgs = X_train[idx]
        
        noise = np.random.normal(0, 1, (half_batch, 100))

        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))#NP ONE CAUSE REAL
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))#NP ZERO CAUSE FAKE
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 


        noise = np.random.normal(0, 1, (batch_size, 100)) 

        valid_y = np.array([1] * batch_size)

    
        g_loss = combined.train_on_batch(noise, valid_y)



        
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        
        if epoch % save_interval == 0:
            save_imgs(epoch)

In [26]:
def save_imgs(epoch):
    r,c=5,5
    noise = np.random.normal(0,1,(r*c,100))
    gen_imgs = generator.predict(noise)
    
    #Rescale the images 0-1
    gen_imgs = 0.5* gen_imgs+0.5
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

In [27]:
optimizer = Adam(0.0002, 0.5)

In [28]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_12 (Dense)            (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-tr

In [34]:
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 256)               25856     
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 256)               0         
                                                                 
 batch_normalization_6 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_18 (Dense)            (None, 512)               131584    
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 512)               0         
                                                                 
 batch_normalization_7 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [35]:
z = Input(shape=(100,))   #Our random input to the generator
img = generator(z)

In [36]:
valid = discriminator(img) 

In [37]:
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [40]:
train(epochs=10000, batch_size=32, save_interval=10)

0 [D loss: 0.719857, acc.: 50.00%] [G loss: 0.459368]
1 [D loss: 0.722531, acc.: 50.00%] [G loss: 0.463320]
2 [D loss: 0.737299, acc.: 50.00%] [G loss: 0.464386]
3 [D loss: 0.742935, acc.: 50.00%] [G loss: 0.457627]
4 [D loss: 0.733867, acc.: 50.00%] [G loss: 0.459687]
5 [D loss: 0.722133, acc.: 50.00%] [G loss: 0.485263]
6 [D loss: 0.725756, acc.: 50.00%] [G loss: 0.466907]
7 [D loss: 0.711360, acc.: 50.00%] [G loss: 0.474118]
8 [D loss: 0.729721, acc.: 50.00%] [G loss: 0.467240]
9 [D loss: 0.740994, acc.: 50.00%] [G loss: 0.462651]
10 [D loss: 0.712923, acc.: 50.00%] [G loss: 0.479438]
11 [D loss: 0.726010, acc.: 50.00%] [G loss: 0.468527]
12 [D loss: 0.737936, acc.: 50.00%] [G loss: 0.483925]
13 [D loss: 0.714295, acc.: 50.00%] [G loss: 0.478031]
14 [D loss: 0.708731, acc.: 50.00%] [G loss: 0.474443]
15 [D loss: 0.724777, acc.: 50.00%] [G loss: 0.484780]
16 [D loss: 0.720203, acc.: 50.00%] [G loss: 0.488435]
17 [D loss: 0.715521, acc.: 50.00%] [G loss: 0.485693]
18 [D loss: 0.706801

149 [D loss: 0.710965, acc.: 50.00%] [G loss: 0.481094]
150 [D loss: 0.705304, acc.: 50.00%] [G loss: 0.489189]
151 [D loss: 0.732669, acc.: 50.00%] [G loss: 0.510089]
152 [D loss: 0.705412, acc.: 50.00%] [G loss: 0.503430]
153 [D loss: 0.695097, acc.: 50.00%] [G loss: 0.464902]
154 [D loss: 0.727360, acc.: 50.00%] [G loss: 0.488887]
155 [D loss: 0.702045, acc.: 50.00%] [G loss: 0.495490]
156 [D loss: 0.656815, acc.: 50.00%] [G loss: 0.495539]
157 [D loss: 0.680900, acc.: 50.00%] [G loss: 0.498632]
158 [D loss: 0.693562, acc.: 50.00%] [G loss: 0.488122]
159 [D loss: 0.736919, acc.: 50.00%] [G loss: 0.486678]
160 [D loss: 0.725746, acc.: 50.00%] [G loss: 0.503468]
161 [D loss: 0.717886, acc.: 50.00%] [G loss: 0.472370]
162 [D loss: 0.705874, acc.: 50.00%] [G loss: 0.487644]
163 [D loss: 0.705503, acc.: 50.00%] [G loss: 0.502187]
164 [D loss: 0.715181, acc.: 50.00%] [G loss: 0.503212]
165 [D loss: 0.701522, acc.: 50.00%] [G loss: 0.497497]
166 [D loss: 0.717766, acc.: 50.00%] [G loss: 0.

297 [D loss: 0.701546, acc.: 53.12%] [G loss: 0.498879]
298 [D loss: 0.703438, acc.: 53.12%] [G loss: 0.471270]
299 [D loss: 0.702368, acc.: 53.12%] [G loss: 0.472801]
300 [D loss: 0.752433, acc.: 50.00%] [G loss: 0.499650]
301 [D loss: 0.703247, acc.: 53.12%] [G loss: 0.510634]
302 [D loss: 0.739977, acc.: 50.00%] [G loss: 0.460337]
303 [D loss: 0.746988, acc.: 50.00%] [G loss: 0.468493]
304 [D loss: 0.687200, acc.: 50.00%] [G loss: 0.488415]
305 [D loss: 0.714316, acc.: 50.00%] [G loss: 0.508845]
306 [D loss: 0.712302, acc.: 50.00%] [G loss: 0.458005]
307 [D loss: 0.687663, acc.: 50.00%] [G loss: 0.467490]
308 [D loss: 0.701563, acc.: 50.00%] [G loss: 0.477162]
309 [D loss: 0.729699, acc.: 50.00%] [G loss: 0.482109]
310 [D loss: 0.721702, acc.: 50.00%] [G loss: 0.502388]
311 [D loss: 0.700818, acc.: 50.00%] [G loss: 0.497180]
312 [D loss: 0.727575, acc.: 50.00%] [G loss: 0.469470]
313 [D loss: 0.709816, acc.: 50.00%] [G loss: 0.491606]
314 [D loss: 0.711831, acc.: 50.00%] [G loss: 0.

445 [D loss: 0.741200, acc.: 50.00%] [G loss: 0.472776]
446 [D loss: 0.750470, acc.: 50.00%] [G loss: 0.468166]
447 [D loss: 0.745666, acc.: 50.00%] [G loss: 0.481916]
448 [D loss: 0.723380, acc.: 50.00%] [G loss: 0.488430]
449 [D loss: 0.736561, acc.: 50.00%] [G loss: 0.488505]
450 [D loss: 0.711595, acc.: 50.00%] [G loss: 0.491531]
451 [D loss: 0.753084, acc.: 50.00%] [G loss: 0.484856]
452 [D loss: 0.721029, acc.: 50.00%] [G loss: 0.481848]
453 [D loss: 0.712954, acc.: 50.00%] [G loss: 0.477463]
454 [D loss: 0.746942, acc.: 50.00%] [G loss: 0.474965]
455 [D loss: 0.776944, acc.: 50.00%] [G loss: 0.472159]
456 [D loss: 0.692536, acc.: 50.00%] [G loss: 0.490001]
457 [D loss: 0.701385, acc.: 50.00%] [G loss: 0.494715]
458 [D loss: 0.739843, acc.: 50.00%] [G loss: 0.466817]
459 [D loss: 0.739482, acc.: 50.00%] [G loss: 0.477677]
460 [D loss: 0.737724, acc.: 50.00%] [G loss: 0.473889]
461 [D loss: 0.735496, acc.: 50.00%] [G loss: 0.473065]
462 [D loss: 0.730191, acc.: 50.00%] [G loss: 0.

593 [D loss: 0.721947, acc.: 50.00%] [G loss: 0.495643]
594 [D loss: 0.657599, acc.: 50.00%] [G loss: 0.488895]
595 [D loss: 0.710750, acc.: 50.00%] [G loss: 0.479679]
596 [D loss: 0.747900, acc.: 50.00%] [G loss: 0.483777]
597 [D loss: 0.703802, acc.: 50.00%] [G loss: 0.498730]
598 [D loss: 0.707151, acc.: 50.00%] [G loss: 0.505386]
599 [D loss: 0.678912, acc.: 50.00%] [G loss: 0.502757]
600 [D loss: 0.670932, acc.: 50.00%] [G loss: 0.511385]
601 [D loss: 0.708476, acc.: 50.00%] [G loss: 0.493312]
602 [D loss: 0.661551, acc.: 50.00%] [G loss: 0.501514]
603 [D loss: 0.692747, acc.: 50.00%] [G loss: 0.495555]
604 [D loss: 0.713931, acc.: 50.00%] [G loss: 0.484861]
605 [D loss: 0.694522, acc.: 50.00%] [G loss: 0.477963]
606 [D loss: 0.737712, acc.: 50.00%] [G loss: 0.493408]
607 [D loss: 0.732147, acc.: 50.00%] [G loss: 0.507721]
608 [D loss: 0.706747, acc.: 50.00%] [G loss: 0.497781]
609 [D loss: 0.690137, acc.: 50.00%] [G loss: 0.499760]
610 [D loss: 0.673899, acc.: 50.00%] [G loss: 0.

...

9399 [D loss: 0.720497, acc.: 50.00%] [G loss: 0.434518]
9400 [D loss: 0.773193, acc.: 50.00%] [G loss: 0.432360]
9401 [D loss: 0.729767, acc.: 50.00%] [G loss: 0.426376]
9402 [D loss: 0.788384, acc.: 50.00%] [G loss: 0.430856]
9403 [D loss: 0.711729, acc.: 50.00%] [G loss: 0.440856]
9404 [D loss: 0.726565, acc.: 50.00%] [G loss: 0.430640]
9405 [D loss: 0.774170, acc.: 50.00%] [G loss: 0.447505]
9406 [D loss: 0.756722, acc.: 50.00%] [G loss: 0.453086]
9407 [D loss: 0.731430, acc.: 50.00%] [G loss: 0.444812]
9408 [D loss: 0.705729, acc.: 50.00%] [G loss: 0.435801]
9409 [D loss: 0.775952, acc.: 50.00%] [G loss: 0.449578]
9410 [D loss: 0.717277, acc.: 50.00%] [G loss: 0.441955]
9411 [D loss: 0.735065, acc.: 50.00%] [G loss: 0.443930]
9412 [D loss: 0.738755, acc.: 50.00%] [G loss: 0.435502]
9413 [D loss: 0.709878, acc.: 50.00%] [G loss: 0.448021]
9414 [D loss: 0.718464, acc.: 50.00%] [G loss: 0.440625]
9415 [D loss: 0.754320, acc.: 50.00%] [G loss: 0.436253]
9416 [D loss: 0.768328, acc.: 5

9543 [D loss: 0.764579, acc.: 50.00%] [G loss: 0.440350]
9544 [D loss: 0.749206, acc.: 50.00%] [G loss: 0.453944]
9545 [D loss: 0.686916, acc.: 53.12%] [G loss: 0.463427]
9546 [D loss: 0.780366, acc.: 46.88%] [G loss: 0.445545]
9547 [D loss: 0.722995, acc.: 50.00%] [G loss: 0.426550]
9548 [D loss: 0.740479, acc.: 50.00%] [G loss: 0.424639]
9549 [D loss: 0.749868, acc.: 50.00%] [G loss: 0.425095]
9550 [D loss: 0.752096, acc.: 50.00%] [G loss: 0.427160]
9551 [D loss: 0.773837, acc.: 50.00%] [G loss: 0.424068]
9552 [D loss: 0.738568, acc.: 50.00%] [G loss: 0.438575]
9553 [D loss: 0.753495, acc.: 50.00%] [G loss: 0.426081]
9554 [D loss: 0.736421, acc.: 50.00%] [G loss: 0.437532]
9555 [D loss: 0.763731, acc.: 50.00%] [G loss: 0.404742]
9556 [D loss: 0.759432, acc.: 50.00%] [G loss: 0.421262]
9557 [D loss: 0.732555, acc.: 50.00%] [G loss: 0.423066]
9558 [D loss: 0.749005, acc.: 50.00%] [G loss: 0.413184]
9559 [D loss: 0.780306, acc.: 50.00%] [G loss: 0.435525]
9560 [D loss: 0.737120, acc.: 5

9687 [D loss: 0.743280, acc.: 50.00%] [G loss: 0.416798]
9688 [D loss: 0.761310, acc.: 50.00%] [G loss: 0.428770]
9689 [D loss: 0.746572, acc.: 50.00%] [G loss: 0.435687]
9690 [D loss: 0.749284, acc.: 50.00%] [G loss: 0.435670]
9691 [D loss: 0.739311, acc.: 50.00%] [G loss: 0.438004]
9692 [D loss: 0.760894, acc.: 50.00%] [G loss: 0.428974]
9693 [D loss: 0.721646, acc.: 50.00%] [G loss: 0.443907]
9694 [D loss: 0.732562, acc.: 50.00%] [G loss: 0.429412]
9695 [D loss: 0.744619, acc.: 50.00%] [G loss: 0.425232]
9696 [D loss: 0.769216, acc.: 50.00%] [G loss: 0.424140]
9697 [D loss: 0.752051, acc.: 50.00%] [G loss: 0.442353]
9698 [D loss: 0.714141, acc.: 50.00%] [G loss: 0.429087]
9699 [D loss: 0.738737, acc.: 50.00%] [G loss: 0.420267]
9700 [D loss: 0.815328, acc.: 50.00%] [G loss: 0.431126]
9701 [D loss: 0.786978, acc.: 50.00%] [G loss: 0.448881]
9702 [D loss: 0.751657, acc.: 50.00%] [G loss: 0.434257]
9703 [D loss: 0.743922, acc.: 50.00%] [G loss: 0.454725]
9704 [D loss: 0.716703, acc.: 5

9831 [D loss: 0.735038, acc.: 50.00%] [G loss: 0.432453]
9832 [D loss: 0.753668, acc.: 50.00%] [G loss: 0.429515]
9833 [D loss: 0.734819, acc.: 50.00%] [G loss: 0.424274]
9834 [D loss: 0.747836, acc.: 50.00%] [G loss: 0.433630]
9835 [D loss: 0.722735, acc.: 50.00%] [G loss: 0.438489]
9836 [D loss: 0.723480, acc.: 50.00%] [G loss: 0.449067]
9837 [D loss: 0.747244, acc.: 50.00%] [G loss: 0.465163]
9838 [D loss: 0.751043, acc.: 50.00%] [G loss: 0.429396]
9839 [D loss: 0.726013, acc.: 50.00%] [G loss: 0.416790]
9840 [D loss: 0.734011, acc.: 50.00%] [G loss: 0.430218]
9841 [D loss: 0.745513, acc.: 50.00%] [G loss: 0.417015]
9842 [D loss: 0.724394, acc.: 50.00%] [G loss: 0.409922]
9843 [D loss: 0.747825, acc.: 50.00%] [G loss: 0.430574]
9844 [D loss: 0.748898, acc.: 50.00%] [G loss: 0.410519]
9845 [D loss: 0.840117, acc.: 50.00%] [G loss: 0.415697]
9846 [D loss: 0.746734, acc.: 50.00%] [G loss: 0.446912]
9847 [D loss: 0.742384, acc.: 50.00%] [G loss: 0.439517]
9848 [D loss: 0.753988, acc.: 5

9975 [D loss: 0.742372, acc.: 50.00%] [G loss: 0.426508]
9976 [D loss: 0.728117, acc.: 50.00%] [G loss: 0.431978]
9977 [D loss: 0.749618, acc.: 50.00%] [G loss: 0.431840]
9978 [D loss: 0.721833, acc.: 50.00%] [G loss: 0.436117]
9979 [D loss: 0.748372, acc.: 50.00%] [G loss: 0.435931]
9980 [D loss: 0.747364, acc.: 50.00%] [G loss: 0.436252]
9981 [D loss: 0.725809, acc.: 50.00%] [G loss: 0.444936]
9982 [D loss: 0.740735, acc.: 50.00%] [G loss: 0.443369]
9983 [D loss: 0.693401, acc.: 50.00%] [G loss: 0.426751]
9984 [D loss: 0.754901, acc.: 50.00%] [G loss: 0.430972]
9985 [D loss: 0.742551, acc.: 50.00%] [G loss: 0.433694]
9986 [D loss: 0.721197, acc.: 50.00%] [G loss: 0.437231]
9987 [D loss: 0.719353, acc.: 50.00%] [G loss: 0.430815]
9988 [D loss: 0.706910, acc.: 50.00%] [G loss: 0.431094]
9989 [D loss: 0.744152, acc.: 50.00%] [G loss: 0.424479]
9990 [D loss: 0.775234, acc.: 50.00%] [G loss: 0.428858]
9991 [D loss: 0.758599, acc.: 50.00%] [G loss: 0.435240]
9992 [D loss: 0.745018, acc.: 5